In [1]:
!pip install polars > istallations.txt

In [2]:
import polars as pl
import numpy as np
import pandas as pd
import os
import time
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/sbermarket-internship-competition/sample_submission.csv
/kaggle/input/sbermarket-internship-competition/train.csv


In [3]:
raw = pl.read_csv('../input/sbermarket-internship-competition/train.csv')
sub = pl.read_csv('../input/sbermarket-internship-competition/sample_submission.csv')

# Приведение столбца 'cart' к типу string
raw = raw.with_columns(raw['cart'].cast(str).alias("cart"))

In [4]:
def t_v_data(raw: pl.DataFrame) -> tuple[pl.DataFrame, pl.DataFrame]:
    """
    Обрабатывает датафрейм polars и возвращает обработынный тренировочный и валидационный датасеты.

    Args:
        raw (pl.DataFrame): Polars DataFrame для обработки.

    Returns:
        tuple[pl.DataFrame, pl.DataFrame]: Обработанные DataFrame для train_data и valid_data.
    """

    # OHE 'cart', группировка по юзеру и дате заказа по максимальным значениям
    # Таким образом для каждой категории, если она была в этот день в заказе у пользователя, будет 1
    train_raw = raw.to_dummies(columns='cart').group_by(['user_id', 'order_completed_at']).max()
    train_raw = train_raw.sort(['user_id', 'order_completed_at'])

    # Вычисление порядкового номера заказа для каждого пользователя
    # pl.lit - создает временный столбец, чтобы вычислять 1 и нумерация начиналась с 0
    train_raw = train_raw.with_columns(
        (pl.col('user_id').cum_count() - pl.lit(1)).over(['user_id']).alias('order_number')
    )

    # Удаление даты. ХОТЯ ТУТ МОЖНО ДОСТАТЬ КАКИЕ_ТО ФИЧИ; НАПРИМЕР ВРЕМЯ СУТОК
    # Можно посмотреть следующее вероятное время заказа
    train_raw = train_raw.drop('order_completed_at')

    # Группировка по 'user_id' и вычисление максимального номера заказа
    grouped = train_raw.group_by('user_id').agg(pl.max('order_number').alias('max_order_number'))

    # Присоединение к train_raw для получения максимального номера заказа для каждого 'user_id'
    merged = train_raw.join(grouped, on='user_id')

    # Определение последнего заказа для каждого пользователя
    last_order = merged['max_order_number'] == merged['order_number']

    # Разделение на train_data (заказы, кроме последнего) и valid_data (последний заказ)
    train_data = train_raw.filter(~last_order).group_by('user_id').sum()
    valid_data = train_raw.filter(last_order)

    return train_data, valid_data

def Train_create(train_data: pl.DataFrame, valid_data: pl.DataFrame) -> pd.DataFrame:
    """
    Обрабатывает train & val и возвращает укомплектованный pd.Dataframe Train.

    Args:
        train_data (pl.DataFrame): DataFrame с данными обучающего набора.
        valid_data (pl.DataFrame): DataFrame с данными валидационного набора.

    Returns:
        pd.DataFrame: Итоговый DataFrame после обработки.
    """
    # Преобразование в длинный формат
    train_melt = train_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='ordered')
    valid_melt = valid_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='target')
    
    Train = train_melt
    # Создание DataFrame с информацией о номере заказа для каждого пользователя в valid_data
    order_number_df = valid_data.select(['user_id', 'order_number']).unique()
    Train = Train.join(order_number_df, on='user_id', how='left')

    # Вычисление рейтинга для каждой записи в Train
    Train = Train.with_columns((pl.col('ordered') / pl.col('order_number')).alias('rating'))

    # Формирование уникального идентификатора для каждой записи в Train
    Train = Train.with_columns((pl.col('user_id').cast(pl.Utf8) + ';' + pl.col('category')).alias('id'))

    # Присоединение целевой переменной из valid_melt к Train
    Train = Train.with_columns(valid_melt.select('target'))

    # Расчет суммарного количества заказанных продуктов по каждой категории в Train
    dl_tmp = Train.group_by('category').agg(pl.col(['ordered']).sum())

    # Присоединение информации о суммарном количестве заказанных продуктов к Train
    Train = Train.join(dl_tmp.select('category', 'ordered'), on='category').rename({"ordered_right": "total_ordered"})

    # Преобразование в pandas DataFrame
    Train = Train.to_pandas()

    return Train

%%time
train_data, val_data = t_v_data(raw)
Train = Train_create(train_data, val_data)


UsageError: Line magic function `%%time` not found.


In [ ]:
Train

In [ ]:
%%time
raw = raw.with_columns(raw['cart'].cast(str).alias("cart"))
train_raw = raw.to_dummies(columns = 'cart')
train_raw = train_raw.group_by(['user_id', 'order_completed_at']).max()
train_raw = train_raw.sort(['user_id', 'order_completed_at'])
train_raw = train_raw.with_columns(
    (pl.col('user_id').cum_count() - pl.lit(1)).over(['user_id']).alias('order_number')
)
train_raw = train_raw.drop('order_completed_at')
grouped = train_raw.group_by('user_id').agg(pl.max('order_number').alias('max_order_number'))
merged = train_raw.join(grouped, on='user_id')
last_order = merged['max_order_number'] == merged['order_number']
train_data = train_raw.filter(~last_order).group_by('user_id').sum()
valid_data = train_raw.filter(last_order)

train_melt = train_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='ordered')
valid_melt = valid_data.drop('order_number').melt(id_vars=['user_id'], variable_name='category', value_name='target')

Train = train_melt

order_number_df = valid_data.select(['user_id', 'order_number']).unique()
Train = Train.join(order_number_df, on='user_id', how='left')
Train = Train.with_columns((pl.col('ordered') / pl.col('order_number')).alias('rating'))
Train = Train.with_columns((pl.col('user_id').cast(pl.Utf8) + ';' + pl.col('category')).alias('id'))
Train = Train.with_columns(valid_melt.select('target'))
dl_tmp = Train.group_by('category').agg(pl.col(['ordered']).sum())
Train = Train.join(dl_tmp.select('category', 'ordered'), on='category').rename({"ordered_right": "total_ordered"})
Train = Train.to_pandas()

In [ ]:
Train

In [ ]:
tmp_pl = valid_melt.select(pl.col(['target', 'user_id']))

In [ ]:
Train = Train.with_columns(valid_melt.select('target'))

In [ ]:
Train = Train.with_columns(valid_melt.select('target'))
dl_tmp = Train.group_by('category').agg(pl.col(['ordered']).sum())
Train = Train.join(dl_tmp.select('category', 'ordered'), on='category').rename({"ordered_right": "total_ordered"})

In [ ]:
%%time
# sparse matrix for temporary use
train_raw = pd.get_dummies(raw_pd, columns = ['cart'], prefix='', prefix_sep='', dtype='bool')
train_raw = train_raw.groupby(['user_id', 'order_completed_at']).any().reset_index()

# order counter for each use
train_raw['order_number'] = train_raw.groupby(['user_id']).cumcount()
train_raw = train_raw.drop('order_completed_at', axis=1)

# separate datasets by the last purchase
last_order = train_raw.groupby(['user_id'])['order_number'].transform(max) == train_raw['order_number']
train = train_raw[~last_order].groupby('user_id').sum().reset_index()
valid = train_raw[last_order].reset_index(drop=True)

#purchase counter for each category for each user
train_melt = pd.melt(train, id_vars=['user_id'], var_name='category', value_name='ordered')
valid_melt = pd.melt(valid, id_vars=['user_id'], var_name='category', value_name='target')

Train = train_melt.copy()

# total purchase counter for each user
order_number = valid[['user_id', 'order_number']].set_index('user_id').squeeze()
Train['orders_total']= Train['user_id'].map(order_number)

#average amount of each category in  customer's purchase
Train['rating'] = Train['ordered'] / Train['orders_total']

# user_id / category as in submission file
Train['id'] = Train['user_id'].astype(str) + ';' + Train['category']

# target variable (the last known purchase)
Train['target'] = valid_melt['target'].astype(int)

#remove those users/categories who are not represented in the submission file
# Train = Train[Train.id.isin(sub.id.unique())].reset_index(drop=True)
#Check
print((Train.sort_values('id')['id'].values == Train.sort_values('id')['id'].values).all())

#purchase counter by all user (for represetned users)
total_ordered = Train.groupby('category')['ordered'].sum()
Train['total_ordered'] = Train['category'].map(total_ordered)

Train

In [ ]:
train